In [1]:
import os
import re
import pickle
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
from urllib.request import urlopen
from io import BytesIO
from PIL import Image
import colour
import numpy as np

In [3]:
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [4]:
cookieFile = "cookie.pkl"
cookies = None

if os.path.exists(cookieFile):
    with open(cookieFile, "rb") as inFile: cookies = pickle.load(inFile)

In [5]:
driver = webdriver.Firefox()
if cookies is not None:
    driver.get("https://weibo.com")
    for co in cookies: driver.add_cookie(co)
driver.get("https://weibo.com")
# _ = input("press ENTER after login to weibo")

In [6]:
# 跳转到搜索“腾讯”官方号
driver.get(r"https://s.weibo.com/user?q=%E8%85%BE%E8%AE%AF&auth=org_vip&page=1")

In [7]:
def wait_for_element(driver, className, timeout=30):
    try:
        el = EC.presence_of_element_located((By.CLASS_NAME, className))
        WebDriverWait(driver, timeout).until(el)
    except TimeoutException:
        print("Timeout")
        return False
    finally:
        return True

In [8]:
def collect_color_temperature(driver):
    url = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "body.S_page"))).value_of_css_property("background-image")
    if "none" in url: return None
    imgurl = re.split('[""]', url)[1]
    image = np.array(Image.open(BytesIO(urlopen(imgurl).read())).convert("RGB"))
    imageXYZ = np.mean(np.mean((image / 255.0), axis=0), axis=0)
    imageXY = colour.XYZ_to_xy(imageXYZ)
    CCT = colour.temperature.xy_to_CCT_McCamy1992(imageXY)
    return CCT

In [9]:
def collect_search(driver, url, num_pages=5):
    driver.get(url)
    result = []
    for i in range(num_pages):
        page = []
        if not wait_for_element(driver, "name"): break
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        for name in soup.select("div.info a.name"):
            page.append((name.get_text(), name["href"]))
        result.append(page)
        try:
            nextPageButton = driver.find_element_by_xpath('//a[text()="下一页"]')
            nextPageButton.click()
        except Exception as e:
            print(e)
            break
    return result

In [10]:
def collect_save(driver,searchData):
    result = []
    for page in searchData:
        for element in page:
            driver.get("https:"+element[1])
            if not wait_for_element(driver, "PCD_counter"): continue
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            eleThree = soup.select("div#plc_main div.PCD_counter td.S_line1")
            if len(eleThree) != 3: continue #确认搜到的是3个element
            row = [
                element[0], # 名字
                "https:"+element[1], # url
                int(eleThree[0].strong.string), # 关注数
                int(eleThree[1].strong.string), # 粉丝数
                int(eleThree[2].strong.string), # 微博数
                collect_color_temperature(driver) # 色温
            ]
            result.append(row)
    result = pd.DataFrame(result,columns = ["名字","url","关注数","粉丝数","微博数","色温"])
    return result

In [11]:
tmp = collect_search(driver, r"https://s.weibo.com/user?q=%E8%85%BE%E8%AE%AF&auth=org_vip&page=1", num_pages=10)

In [12]:
data = collect_save(driver, tmp)

Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout


In [13]:
data.to_csv("sina.csv", index=False, encoding='utf-8-sig')

In [14]:
data

,名字,url,关注数,粉丝数,微博数,色温
0,腾讯视频,https://weibo.com/ilovevideo,3191,14513280,97171,6.006279e+03
1,腾讯体育,https://weibo.com/tengxuntiyu,391,3357631,48679,2.506354e+03
2,腾讯微信团队,https://weibo.com/weixinteam,44,576102,1188,NaN
3,腾讯视频VIP,https://weibo.com/hollywoodtheatre,804,2053315,29705,5.995864e+03
4,腾讯综艺,https://weibo.com/remendezongyi,1713,4148991,40310,7.063860e+03
...,...,...,...,...,...,...
166,腾讯新闻我的时代和我,https://weibo.com/u/6505340726,22,9155,163,5.810269e+03
167,饭局的诱惑,https://weibo.com/u/5934477397,159,107732,1364,1.756541e+07
168,极品飞车OL官博,https://weibo.com/u/5896258942,102,75734,729,7.665118e+03
169,腾讯漫动画,https://weibo.com/u/6863132053,81,57130,627,2.888846e+03


In [15]:
driver.get("https://weibo.com")
with open(cookieFile, "wb") as outFile:
    pickle.dump(driver.get_cookies(), outFile)

driver.close()